<a href="https://colab.research.google.com/github/ErickMartinezz/ProcesamientodelHabla/blob/main/Desafio_3_ErickMartinez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
cossim

array([0.1382319 , 0.1067036 , 0.23029327, ..., 0.12320753, 0.08765353,
       0.04415046])

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 2

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.



Respuesta 1

In [39]:
import random
import numpy as np
import textwrap
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




In [40]:
# Cargar datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
X_train_text = newsgroups_train.data
y_train = newsgroups_train.target
target_names = newsgroups_train.target_names

# Vectorizar
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=5, ngram_range=(1, 2))
X_train = vectorizer.fit_transform(X_train_text)

In [34]:
# Semilla y selección de documentos
random
doc_azar = random.sample(range(len(X_train_text)), 5)
doc_azar

[1424, 9674, 6912, 520, 488]

In [46]:
# Similaridad de documentos
for idx in doc_azar:
    print("=" * 100)
    print(f"DOCUMENTO: {idx} | Clase: {target_names[y_train[idx]]}")
    print("-" * 100)
    print(textwrap.fill(X_train_text[idx][:500], width=100))

    # Similaridad coseno
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    cossim[idx] = 0  # Excluir el documento original

     # Top 5 documentos similares
    most_similar = np.argsort(cossim)[::-1][:5]
    print("\nDOCUMENTOS MÁS SIMILARES:")
    for sim_idx in most_similar:
        sim_score = cossim[sim_idx] * 100
        print("-" * 100)
        print(f"Índice: {sim_idx} | Clase: {target_names[y_train[sim_idx]]} | Similitud: {sim_score:.2f}%")
        print(textwrap.fill(X_train_text[sim_idx][:300], width=100))
    print("=" * 100 + "\n")

DOCUMENTO: 1424 | Clase: rec.sport.hockey
----------------------------------------------------------------------------------------------------
 Hopefully, a miracle (o.k. not quite a miracle, but close!) will occur and Pittsburgh will be
elminated prior to the finals.  If they make it again,  they will probably keep the Cup.  If they
don't, it's the Hawks' turn!  GO BLACKHAWKS!!  CONGRATS TO JEREMY ROENICK FOR BEING ONLY THE 2ND
HAWKS PLAYER TO POST  BACK TO BACK 50 GOAL SEASONS!!

DOCUMENTOS MÁS SIMILARES:
----------------------------------------------------------------------------------------------------
Índice: 8689 | Clase: rec.sport.hockey | Similitud: 38.22%
The Hawks win!!  Jermey Roenick scored his 50 th goal and the Hawks put the Leafs in their place,
the losers column.  If the Leafs can not even beat the Hawks in a match that had little or no
meaning I will hate to see them against the Wings.    Oh btw I laugh at rm, that jerky!!!  GO
HAWKS!!!
--------------------------------


**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

Respuesta 2

In [52]:
# MultinomialNB
clf_mnb = MultinomialNB()
clf_mnb.fit(X_train, y_train)
y_pred_mnb = clf_mnb.predict(X_test)
print("\nF1-score macro (MultinomialNB):", f1_score(y_test, y_pred_mnb, average='macro'))



F1-score macro (MultinomialNB): 0.6453501281993387


In [53]:
# ComplementNB
clf_cnb = ComplementNB()
clf_cnb.fit(X_train, y_train)
y_pred_cnb = clf_cnb.predict(X_test)
print("F1-score macro (ComplementNB):", f1_score(y_test, y_pred_cnb, average='macro'))

F1-score macro (ComplementNB): 0.6881187655621652




**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

Respuesta 3

In [57]:
# Asegurar que el vocabulario sea el correcto
X_td = X_train.transpose()
idx2word = {v: k for k, v in vectorizer.vocabulary_.items()}

# Selección de palabras
selected_words = ['technology', 'computer', 'science', 'network', 'algorithm']

# Verificar si las palabras seleccionadas están en el vocabulario
for word in selected_words:
    if word not in vectorizer.vocabulary_:
        print(f"Advertencia: la palabra '{word}' no está en el vocabulario.")
    else:
        print(f"La palabra '{word}' está en el vocabulario.")

# Buscar los índices de las palabras en el vocabulario
word_indices = [vectorizer.vocabulary_[w] for w in selected_words if w in vectorizer.vocabulary_]

# Calcular similitud
for idx in word_indices:
    sim = cosine_similarity(X_td[idx], X_td)[0]
    top_similar = np.argsort(sim)[::-1][1:6]
    print(f"\nPalabra: {idx2word[idx]}")
    for i in top_similar:
        print(f" -> {idx2word[i]} (sim: {sim[i]:.3f})")


La palabra 'technology' está en el vocabulario.
La palabra 'computer' está en el vocabulario.
La palabra 'science' está en el vocabulario.
La palabra 'network' está en el vocabulario.
La palabra 'algorithm' está en el vocabulario.

Palabra: technology
 -> use technology (sim: 0.215)
 -> technology available (sim: 0.207)
 -> science technology (sim: 0.202)
 -> new technology (sim: 0.197)
 -> encryption technology (sim: 0.193)

Palabra: computer
 -> computer science (sim: 0.241)
 -> computer graphics (sim: 0.217)
 -> turn computer (sim: 0.168)
 -> new computer (sim: 0.166)
 -> use computer (sim: 0.151)

Palabra: science
 -> computer science (sim: 0.372)
 -> scientific (sim: 0.327)
 -> science collection (sim: 0.258)
 -> like science (sim: 0.240)
 -> behaviorists (sim: 0.240)

Palabra: network
 -> products using (sim: 0.185)
 -> network card (sim: 0.185)
 -> novell (sim: 0.173)
 -> utilizing (sim: 0.171)
 -> windows workgroups (sim: 0.167)

Palabra: algorithm
 -> encryption algorithm (sim

En este tercer punto use las palabras technology, computer, science, network y algorithm.
Viendo la comparacion con otras palabras entiendo que tienen buena similitud, excepto la palabra network que seria la mas baja.